# Blind Spot Detection (BSD) with Active Intervention

*Project for the course Smart Vehicular Systems (Academic Year 2024/2025).*

**Name:** Pablo Sebastian

**Surname:** Vargas Grateron

**Email:** pablo.vargasgrateron@studio.unibo.it

This project implements a blind spot detection system using [CARLA simulator](https://carla.org/) (Version 0.9.15) and MQTT for communication. The system detects vehicles in the blind spot of a simulated car and can trigger an active intervention if necessary.

> Before running the code, ensure that the CARLA server is running and the MQTT broker is accessible.

## Libraries and setup

To execute the code, you need to install the libraries listed in the `requirements.txt` file. You can install them using:
```bash
pip install -r requirements.txt
```

For the local imports, ensure that the `manual_control.py` and `manual_control_steeringwheel.py` files are in the same directory as this notebook or adjust the import path accordingly.

> The local imports can be found in the CARLA repository under the `PythonAPI/examples` directory.

In [ ]:
import carla, time, pygame, math, random, cv2, os
import numpy as np
import paho.mqtt.client as mqtt
from ultralytics import YOLO

# Local imports
import manual_control
import manual_control_steeringwheel

This cell configures the CARLA server connection.

In [ ]:
carla_client = carla.Client('localhost', 2000)
carla_client.set_timeout(10.0)

world = carla_client.get_world()
spectator = world.get_spectator()

This cell configures the MQTT client. We will use the public broker `test.mosquitto.org` for testing purposes. In a production environment, you should use a secure and private MQTT broker.

In [ ]:
BROKER = "test.mosquitto.org"
PORT = 1883
TOPIC = "svs_blind_spot_detection"

mqtt_client = mqtt.Client()
mqtt_client.connect(BROKER, PORT, 60)

Further setup for the MQTT client, including connection and message handling.

In [ ]:
def mqtt_on_connect(client, userdata, flags, rc):
    """
    Callback function triggered when MQTT client connects to broker
    - Automatically subscribes to the BSD topic once connected
    - rc codes: 0=success, 1=wrong protocol, 2=invalid client ID, etc.
    """
    print(f"[MQTT] Connected (code {rc})")
    client.subscribe(TOPIC)

def mqtt_on_message(client, userdata, message):
    """
    Callback function triggered when a message is received on subscribed topics
    - Decodes incoming messages and prints them
    """
    decoded_message = message.payload.decode()
    print(f"[MQTT] Message on {message.topic} -> {decoded_message}")

def mqtt_send_message(topic, message):
    """
    Send a message to specified MQTT topic
    - Used to publish BSD alerts, status updates, or intervention commands
    - Message format: string (will be encoded to bytes automatically)
    """
    mqtt_client.publish(topic, message)
    print(f"[MQTT] Sent message: {message} to topic: {topic}")

mqtt_client.on_connect = mqtt_on_connect
mqtt_client.on_message = mqtt_on_message    

mqtt_client.loop_start()

## Functions

### CARLA basic functions (From the laboratory)

In [ ]:
def move_spectator_to(transform, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance
    
    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll
    
    spectator_transform = carla.Transform(back_location, transform.rotation)
    
    spectator.set_transform(spectator_transform)

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def draw_on_screen(world, transform, content='O', color=carla.Color(0, 255, 0), life_time=20):
    world.debug.draw_string(transform.location, content, color=color, life_time=life_time)

### Computer vision functions

We will implement functions to capture images from the car's cameras and detect vehicles using [YOLOv11](https://docs.ultralytics.com/models/yolo11/) model. We will use the nano version for real-time processing.

> Make sure to download the YOLOv11 nano model and place it in the `cv_model` directory (Or change the path in the code below).

In [ ]:
MODEL_DIR = "cv_model"
MODEL_NAME = "yolo11n.pt"

cv_model_path = os.path.join(MODEL_DIR, MODEL_NAME)
cv_model = YOLO(cv_model_path)

def detect_vehicles(image, model=cv_model, conf=0.25):
    """
    Detect vehicles in an image using YOLO model for blind spot detection
    
    Args:
        image: NumPy array representing RGB image from camera
        model: YOLOv11 model object (default: pre-loaded cv_model)
        conf: Confidence threshold for detections (default: 0.25 = 25%)
    
    Returns:
        list: List of vehicle detections, each containing:
            - xmin, ymin, xmax, ymax: Bounding box coordinates in pixels
            - confidence: Detection confidence score (0.0 to 1.0)
            - class_id: YOLO class ID number
            - label: Vehicle type ('car', 'truck', 'bus', 'motorcycle')
    """
    results = model(image, conf=conf)
    detections = []
    
    for result in results:
        if result.boxes is not None:
            for box in result.boxes:
                label = model.names[int(box.cls[0])]
                # Filter for vehicle classes (car, truck, bus, motorcycle)
                if box.conf[0] > conf and label in ['car', 'truck', 'bus', 'motorcycle']:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    detection = {
                        'xmin': int(x1),
                        'ymin': int(y1),
                        'xmax': int(x2),
                        'ymax': int(y2),
                        'confidence': float(box.conf[0]),
                        'class_id': int(box.cls[0]),
                        'label': label
                    }
                    detections.append(detection)
    
    return detections

def carla_image_to_array(image):
    """
    Convert CARLA raw image data to NumPy RGB array for YOLO processing
    
    Args:
        image: CARLA camera sensor image object with raw_data attribute
    
    Returns:
        numpy.ndarray: RGB image array (height, width, 3) suitable for YOLO
    """
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3]
    array = array[:, :, [2, 1, 0]] # Convert BGR to RGB
    return array

def process_camera_image(image):
    """
    Complete pipeline: Convert CARLA image and detect vehicles for BSD system
    
    Args:
        image: CARLA camera sensor image object
    
    Returns:
        tuple: (rgb_array, detections)
            - rgb_array: Converted RGB image as NumPy array
            - detections: List of vehicle detections from YOLO
    """
    rgb_array = carla_image_to_array(image)
    detections = detect_vehicles(rgb_array)
    
    if detections:
        print(f"Detected {len(detections)} vehicles:")
        for det in detections:
            print(f"  - {det['label']}: {det['confidence']:.2f} confidence")
    
    return rgb_array, detections

### Sensors functions

The next cells implement the functions to spawn:

- The **camera sensors** to capture image to further process with YOLOv5 and detect vehicles.
- The **radar sensor** to detect vehicles in the blind spot.

In [ ]:
def spawn_camera(attach_to=None, transform=carla.Transform(carla.Location(x=0.0, y=0.0, z=1.0), carla.Rotation(pitch=0)), width=800, height=600):
    """
    Spawn an RGB camera sensor in the CARLA world for vehicle detection
    
    Args:
        attach_to: Vehicle or object to attach camera to (None = spawn freely)
        transform: Camera position and orientation relative to attachment point
        width: Image width in pixels (default: 800)
        height: Image height in pixels (default: 600)
    
    Returns:
        carla.Sensor: RGB camera sensor object for capturing images
    """
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(width))
    camera_bp.set_attribute('image_size_y', str(height))
    camera = world.spawn_actor(camera_bp, transform, attach_to=attach_to)
    return camera

def spawn_radar(attach_to=None, transform=carla.Transform(carla.Location(x=0.0, y=0.0, z=1.0), carla.Rotation(pitch=0)), range=100, horizontal_fov=30, vertical_fov=30):
    """
    Spawn a radar sensor in the CARLA world for object detection
    
    Args:
        attach_to: Vehicle or object to attach radar to (None = spawn freely)
        transform: Radar position and orientation relative to attachment point
        range: Detection range in meters (default: 100m)
        horizontal_fov: Horizontal field of view in degrees (default: 30°)
        vertical_fov: Vertical field of view in degrees (default: 30°)
    
    Returns:
        carla.Sensor: Radar sensor object for detecting nearby objects
    """
    radar_bp = world.get_blueprint_library().find('sensor.other.radar')
    radar_bp.set_attribute('horizontal_fov', str(horizontal_fov))
    radar_bp.set_attribute('vertical_fov', str(vertical_fov))
    radar_bp.set_attribute('range', str(range))
    radar = world.spawn_actor(radar_bp, transform, attach_to=attach_to)
    return radar

## Main program and code execution

### World setup

### Car setup

### Main loop